<a href="https://colab.research.google.com/github/shivangibithel/IRMiDis_Task2/blob/main/Text_Classification_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[A Comprehensive Guide to Understand and Implement Text Classification in Python](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)

# Imports

In [8]:
# pip install emoji --upgrade

In [9]:
import numpy as np
import pandas as pd #to work with csv files

#matplotlib imports are used to plot confusion matrices for the classifiers
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 

#import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

#pre-processing of text
import string
import re

#import classifiers from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

#import different metrics to evaluate the classifiers
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn import metrics

#import time function from time module to track the training duration
from time import time

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [10]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Section 1: Load and explore the dataset

In [17]:
our_data = pd.read_csv("cleaned_train.csv",index_col = "id")
test_data = pd.read_csv("cleaned_test.csv",index_col = "id")
our_data = shuffle(our_data)
our_data.head()

,tweet,label,label_num
id,,,
1.326010e+18,fyi realdonaldtrumpthat pfizer #coronavirus va...,Neutral,2
1.325730e+18,dr carrie madej warns about coronavirus vaccin...,AntiVax,1
1.326220e+18,absolutely not everyone has bodily autonomy va...,AntiVax,1
1.326230e+18,covid vaccine is hereeeeeeee,ProVax,0
1.325770e+18,fantastic news pfizer says early analysis show...,ProVax,0


In [18]:
display(our_data.shape) #Number of rows (instances) and columns in the dataset
print(our_data["label"].value_counts()/our_data.shape[0]) #Class distribution in the dataset

(2792, 3)

Neutral    0.361748
ProVax     0.354943
AntiVax    0.283309
Name: label, dtype: float64


In [19]:
# Prepare Dataset
trainDF = pd.DataFrame()
trainDF['tweet'] = our_data.tweet
trainDF['label'] = our_data.label

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['tweet'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
# train_y = encoder.fit_transform(trainDF['label'])
valid_y = encoder.fit_transform(valid_y)

In [20]:
trainDF

,tweet,label
id,,
1.326010e+18,fyi realdonaldtrumpthat pfizer #coronavirus va...,Neutral
1.325730e+18,dr carrie madej warns about coronavirus vaccin...,AntiVax
1.326220e+18,absolutely not everyone has bodily autonomy va...,AntiVax
1.326230e+18,covid vaccine is hereeeeeeee,ProVax
1.325770e+18,fantastic news pfizer says early analysis show...,ProVax
...,...,...
1.325770e+18,such a groundbreaking day finally after nearly...,ProVax
1.333420e+18,vespasian mongibellus davekeating very nice bu...,Neutral
1.325860e+18,will fordnation botch the covid vaccine like t...,AntiVax


### Section 2: Text Pre-processing

Typical steps involve tokenization, lower casing, removing, stop words, punctuation markers etc, and vectorization. Other processes such as stemming/lemmatization can also be performed. Here, we are performing the following steps: removing br tags, punctuation, numbers, and stopwords. While we are using sklearn's list of stopwords, there are several other stop word lists (e.g., from NLTK) or sometimes, custom stopword lists are needed depending on the task. 

In [21]:
stopwords = stop_words.ENGLISH_STOP_WORDS
def clean(doc): #doc is a string of text
    # doc = doc.replace("@", " ")
    # doc = doc.replace(" ", " ")
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    # doc = " ".join([token for token in doc.split() if token not in stopwords])
    #remove punctuation and numbers
    return doc

In [22]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [23]:
def remove_url(data):
  url = re.compile("http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+", re.UNICODE)
  return re.sub(url, '', data)

In [24]:
# for i in range(len(our_data)):
#   our_data.tweet.iloc[i] = remove_url(our_data.tweet.iloc[i])
#   our_data.tweet.iloc[i] = clean(our_data.tweet.iloc[i])
#   our_data.tweet.iloc[i] = remove_emojis(our_data.tweet.iloc[i])

In [25]:
our_data.head()

,tweet,label,label_num
id,,,
1.326010e+18,fyi realdonaldtrumpthat pfizer #coronavirus va...,Neutral,2
1.325730e+18,dr carrie madej warns about coronavirus vaccin...,AntiVax,1
1.326220e+18,absolutely not everyone has bodily autonomy va...,AntiVax,1
1.326230e+18,covid vaccine is hereeeeeeee,ProVax,0
1.325770e+18,fantastic news pfizer says early analysis show...,ProVax,0


# Section 3: Modeling

Now we are ready for the modelling. We are going to use algorithms from sklearn package. We will go through the following steps:

1 Read train data    
2 Extract features from the training data using CountVectorizer, which is a bag of words feature  implementation. We will use the pre-processing function above in conjunction with Count Vectorizer  
3 Transform the test data into the same feature vector as the training data.  
4 Train the classifier  
5 Evaluate the classifier  

# Feature Engineering

In [26]:
X_train = train_x
# X_train = trainDF['tweet']
y_train =train_y
print(X_train.shape, y_train.shape, valid_x.shape, valid_y.shape)

(2094,) (2094,) (698,) (698,)


In [27]:
X_train

id
1.326600e+18    fighting stigma govt has a plan to ensure covi...
1.335960e+18    mrna long the canadian government has doubled ...
1.329440e+18    astrazeneca vaccine shows strong immune respon...
1.325790e+18    do they think we are fucking stupid just days ...
1.325800e+18    donaldjtrumpjr all pharmaceutical companies ar...
                                      ...                        
1.330910e+18    what are the potential #supplychain challenges...
1.325920e+18    tomfole spduckworth camestewart i think a mand...
1.326080e+18    me i refuse to take the mark of the beast covi...
1.328060e+18    govmikedewine and we need rapid home frequent ...
1.325840e+18    never related to a tweet so much in my life fu...
Name: tweet, Length: 2094, dtype: object

 ### Count Vectors as features

In [28]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['tweet'])

# transform the training and validation data using count vectorizer object
X_train_dtm =  count_vect.transform(X_train)
xvalid_count =  count_vect.transform(valid_x)

In [29]:
print(X_train_dtm.shape, xvalid_count.shape)

(2094, 7716) (698, 7716)


### TF-IDF Vectors as features

In [30]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['tweet'])
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
print(xtrain_tfidf.shape, xvalid_tfidf.shape)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['tweet'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
print(xtrain_tfidf_ngram.shape, xvalid_tfidf_ngram.shape)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['tweet'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 
print(xtrain_tfidf_ngram_chars.shape, xvalid_tfidf_ngram_chars.shape)

(2094, 5000) (698, 5000)
(2094, 5000) (698, 5000)
(2094, 5000) (698, 5000)


In [31]:
print(xtrain_tfidf_ngram)

  (0, 4155)	0.20712033129118507
  (0, 4066)	0.3042002761324843
  (0, 2962)	0.3233232635365891
  (0, 1666)	0.28507728872837956
  (0, 995)	0.20235232602790756
  (0, 915)	0.3560142264282778
  (0, 180)	0.336891239024173
  (0, 178)	0.2609851738370408
  (0, 155)	0.27561303864875997
  (0, 75)	0.3560142264282778
  (0, 74)	0.3560142264282778
  (1, 4397)	0.22733882991202398
  (1, 4099)	0.28984530578623174
  (1, 4098)	0.28984530578623174
  (1, 4033)	0.1561104810608021
  (1, 3135)	0.22453931344602923
  (1, 3134)	0.22192054068412514
  (1, 3104)	0.3062978206920625
  (1, 2744)	0.2972433483571516
  (1, 2380)	0.28984530578623174
  (1, 2372)	0.3062978206920625
  (1, 2368)	0.2430731509456039
  (1, 1631)	0.2972433483571516
  (1, 1283)	0.2835903499173156
  (1, 952)	0.2475863418046144
  :	:
  (2091, 1699)	0.23633778910428332
  (2091, 921)	0.05068763119162931
  (2091, 729)	0.17883143638795057
  (2092, 4690)	0.26161637661656845
  (2092, 4318)	0.2167005761515349
  (2092, 4170)	0.30690369765155834
  (2092, 4155

### Word Embeddings  --> Not Working

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/MyDrive/

Can also use pre-train embeddings from Gensim library in future

In [52]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(our_data['tweet'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Text / NLP based features

In [ ]:
trainDF = our_data
trainDF['char_count'] = trainDF['tweet'].apply(len)
trainDF['word_count'] = trainDF['tweet'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['tweet'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['tweet'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['tweet'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt
trainDF['noun_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['tweet'].apply(lambda x: check_pos_tag(x, 'pron'))

### Topic Models as features

In [ ]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(X_train_dtm)
topic_word = lda_model.components_ 
vocab = vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [ ]:
topic_summaries

# Classifiers

### Non Deep Methods

In [190]:
# Naive Bayes Classifier
nb = MultinomialNB() #instantiate a Multinomial Naive Bayes model
nb.fit(X_train_dtm, y_train)#train the count vectorizer model
# nb.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# nb.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# nb.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [201]:
# Logistic Regression Classifier
logreg = LogisticRegression(class_weight="balanced",max_iter=1000) #instantiate a logistic regression model
# logreg.fit(X_train_dtm, y_train) #fit the model with training data
logreg.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# logreg.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# logreg.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [212]:
# Linear SVM Classifier
classifier = LinearSVC(class_weight='balanced') #instantiate a logistic regression model
# classifier.fit(X_train_dtm, y_train) #fit the model with training data
classifier.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# classifier.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# classifier.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [222]:
# Bagging Model
classifier = ensemble.RandomForestClassifier()
classifier.fit(X_train_dtm, y_train)
# classifier.fit(xtrain_tfidf, y_train)#train the TF-IDF model
# classifier.fit(xtrain_tfidf_ngram, y_train)#train the TF-IDF ngram model
# classifier.fit(xtrain_tfidf_ngram_chars, y_train)#train the TF-IDF ngram CHAR model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [229]:
# Extereme Gradient Boosting
EXB = xgboost.XGBClassifier()
EXB.fit(X_train_dtm.tocsc(), y_train)
# EXB.fit(xtrain_tfidf.tocsc(), y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [248]:
# Shallow Neural Network
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf.shape[1])
classifier.fit(xtrain_tfidf.toarray(), y_train,epochs =1, shuffle = True)

66/66 [==============================] - 1s 4ms/step - loss: 0.5355


### Deep Neural Models

In [53]:
# CNN
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)
    
    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
classifier.fit(train_seq_x, train_y)
# train_seq_x -- > Word Embeddings required -->Later
# accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
# print "CNN, Word Embeddings",  accuracy

66/66 [==============================] - 3s 26ms/step - loss: 0.0742


In [54]:
train_seq_x.shape

(2094, 70)

In [55]:
#  LSTM
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier_lstm = create_rnn_lstm()
classifier_lstm.fit(train_seq_x, train_y)
# train_seq_x -- > Word Embeddings required -->Later

66/66 [==============================] - 8s 88ms/step - loss: -0.0712


In [56]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier_rnn = create_rnn_gru()
classifier_rnn.fit(train_seq_x, train_y)
# train_seq_x -- > Word Embeddings required -->Later

66/66 [==============================] - 7s 74ms/step - loss: -0.0862


In [57]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier_brnn = create_bidirectional_rnn()
classifier_brnn.fit(train_seq_x, train_y)
# train_seq_x -- > Word Embeddings required -->Later

66/66 [==============================] - 12s 129ms/step - loss: -0.3691


In [58]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier_rcnn = create_rcnn()
classifier_rcnn.fit(train_seq_x, train_y)
# train_seq_x -- > Word Embeddings required -->Later

66/66 [==============================] - 3s 27ms/step - loss: -0.1242


# Evaluation

### Naive Bayes Evaluation

In [185]:
predictions = nb.predict(xvalid_count)
print(" NB, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, Count Vectors: 0.7320916905444126


In [187]:
predictions = nb.predict(xvalid_tfidf)
print(" NB, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 NB, WordLevel TF-IDF: 0.7134670487106017


In [189]:
predictions = nb.predict(xvalid_tfidf_ngram)
print(" NB, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, N-Gram Vectors: 0.6762177650429799


In [191]:
predictions = nb.predict(xvalid_tfidf_ngram_chars)
print(" NB, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 NB, CharLevel Vectors: 0.673352435530086


### Linear Classification --> Logistic Regression

In [194]:
predictions = logreg.predict(xvalid_count)
print(" LR, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, Count Vectors: 0.7134670487106017


In [196]:
predictions = logreg.predict(xvalid_tfidf)
print(" LR, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 LR, WordLevel TF-IDF: 0.7306590257879656


In [198]:
predictions = logreg.predict(xvalid_tfidf_ngram)
print(" LR, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, N-Gram Vectors: 0.6776504297994269


In [200]:
predictions = logreg.predict(xvalid_tfidf_ngram_chars)
print(" LR, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 LR, CharLevel Vectors: 0.7048710601719198


### SVM Evaluation

In [204]:
predictions = classifier.predict(xvalid_count)
print(" SVM, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, Count Vectors: 0.7020057306590258


In [213]:
predictions = classifier.predict(xvalid_tfidf)
print(" SVM, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 SVM, WordLevel TF-IDF: 0.7392550143266475


In [209]:
predictions = classifier.predict(xvalid_tfidf_ngram)
print(" SVM, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, N-Gram Vectors: 0.663323782234957


In [211]:
predictions = classifier.predict(xvalid_tfidf_ngram_chars)
print(" SVM, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 SVM, CharLevel Vectors: 0.7177650429799427


### Random Forrest Evaluation Evaluation

In [224]:
predictions = classifier.predict(xvalid_count)
print(" RF, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, Count Vectors: 0.6934097421203438


In [215]:
predictions = classifier.predict(xvalid_tfidf)
print(" RF, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 RF, WordLevel TF-IDF: 0.6905444126074498


In [219]:
predictions = classifier.predict(xvalid_tfidf_ngram)
print(" RF, N-Gram Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, N-Gram Vectors: 0.6532951289398281


In [221]:
predictions = classifier.predict(xvalid_tfidf_ngram_chars)
print(" RF, CharLevel Vectors:", metrics.accuracy_score(predictions, valid_y))

 RF, CharLevel Vectors: 0.6432664756446992


### Boosting

In [226]:
predictions = EXB.predict(xvalid_count.tocsc())
print(" EXB, Count Vectors:", metrics.accuracy_score(predictions, valid_y))

 EXB, Count Vectors: 0.6977077363896849


In [228]:
predictions = EXB.predict(xvalid_tfidf.tocsc())
print(" RF, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 RF, WordLevel TF-IDF: 0.6805157593123209


### Shallow NN

In [249]:
predictions = classifier.predict(xvalid_tfidf)
predictions = predictions.argmax(axis=-1)
print(" Shallow NN, WordLevel TF-IDF:", metrics.accuracy_score(predictions, valid_y))

 Shallow NN, WordLevel TF-IDF: 0.27793696275071633


### Deep  --> poor results due to embeddings from different domain
Need better pre-trained embeddings

In [59]:
predictions = classifier.predict(valid_seq_x)
predictions = predictions.argmax(axis=-1)
print(" CNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

 CNN, WordEmbeddings: 0.27507163323782235


In [60]:
predictions = classifier_lstm.predict(valid_seq_x)
predictions = predictions.argmax(axis=-1)
print(" LSTM, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

 LSTM, WordEmbeddings: 0.27507163323782235


In [61]:
predictions = classifier_rnn.predict(valid_seq_x)
predictions = predictions.argmax(axis=-1)
print(" RNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

 RNN, WordEmbeddings: 0.27507163323782235


In [62]:
predictions = classifier_brnn.predict(valid_seq_x)
predictions = predictions.argmax(axis=-1)
print(" B RNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

 B RNN, WordEmbeddings: 0.27507163323782235


In [63]:
predictions = classifier_brnn.predict(valid_seq_x)
predictions = predictions.argmax(axis=-1)
print(" RCNN, WordEmbeddings:", metrics.accuracy_score(predictions, valid_y))

 RCNN, WordEmbeddings: 0.27507163323782235
